In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [20]:
url = 'http://kims4all.kr/login.aspx?ReturnUrl=%2fAPPFJ%2fDefault.aspx'

# 정리하려는 상식 중 가장 최신 아이템 페이지의 웹주소를 확인하면 됩니다.
# 예를들어, 'http://kims4all.kr/APPFJ/MediaTest/View.aspx?Num=5988&BoardName=generalKnowledge' 이곳의 Num=다음의 숫자를 입력하면 됩니다. 

start_num=int(input('시작 번호를 입력하세요(큰 수)')) # 예시:5978
stop_num=int(input('종료 번호를 입력하세요(작은 수)')) # 예시:5969
target_num=start_num 
scrap_count = 0
remaining_count = start_num - stop_num + 1
error_count = 0

title_list=[]
definition_list=[]
example_list=[]  
note_list=[]

month = input('월을 입력하세요') # 예시: 6
week = input('주를 입력하세요') # 예시: 1


In [21]:
driver = webdriver.Chrome(executable_path=r'C:\Users\judya\Desktop\취준\chromedriver_win32\chromedriver.exe')
driver.get(url)

time.sleep(1)

# 로그인
driver.find_element_by_name("Login1$UserName").send_keys('박윤주')
driver.find_element_by_name("Login1$Password").send_keys('1234')
driver.find_element_by_name('Login1$LoginButton').click()
time.sleep(1)

# 스크래핑 
while scrap_count < (start_num - stop_num + 1 ): 

    scrap_count += 1
    remaining_count -=1 
    
    driver.get(f'http://kims4all.kr/APPFJ/MediaTest/View.aspx?Num={target_num}&BoardName=generalKnowledge')
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # 표제어 가져오기
    title = soup.select_one('#ctl00_ContentPlaceHolder1_lblWordNPhrase').get_text()
    
    if title == '':
        
        error_count +=1
        print(f'{error_count}번째 삭제된 페이지 발견: {target_num}')
        target_num -=1
        
        continue

    title_list.append(title) 

    # 정의 가져오기 
    definition = soup.select_one('#ctl00_ContentPlaceHolder1_lblSentence').get_text()
    definition_list.append(definition) 

    # 예문 가져오기 
    example = soup.select_one('#ctl00_ContentPlaceHolder1_lblNote').get_text()
    example_list.append(example)

    # 특기사항 가져오기 
    note = soup.select_one('#ctl00_ContentPlaceHolder1_lblQuestions').get_text()
    note_list.append(note) 
    
    target_num -=1
        
    if scrap_count%5 == 0 :
            print(f'{scrap_count}번째 상식 정리 완료... 남은 상식 {remaining_count}개')
    
print(f'상식 정리 완료. 정리된 상식 총 {len(title_list)}개, 삭제된 페이지 {error_count}개')
    

5번째 상식 정리 완료... 남은 상식 5개
1번째 삭제된 페이지 발견: 5970
10번째 상식 정리 완료... 남은 상식 0개
상식 정리 완료. 정리된 상식 총 9개, 삭제된 페이지 1개


In [22]:
from docx import Document 
from docx.shared import Pt
from docx2pdf import convert

# install 방법: prompt에서 pip install python-docx
# pip install docx2pdf

In [24]:
document = Document()
doc_num=0

style = document.styles['Normal']
style.font.name = '맑은 고딕'
style.font.size = Pt(10)

for i in range(len(title_list)):

    
    paragraph=document.add_paragraph(f'{doc_num+1}. '+title_list[doc_num])
    document.add_paragraph(definition_list[doc_num])
    
    if example_list[doc_num] != '':
        document.add_paragraph(example_list[doc_num])
    
    if note_list[doc_num] !='':
        document.add_paragraph(note_list[doc_num])
    document.add_paragraph()
    
    doc_num +=1

document.save(f'아프지 {month}월 {week}주차 시사상식.docx')

# 경로지정
input_file = 'C:\\Users\\judya\\Desktop\\취준\\상식\\아프지\\' + f'아프지 {month}월 {week}주차 시사상식.docx'
output_file = 'C:\\Users\\judya\\Desktop\\취준\\상식\\아프지\\' + f'아프지 {month}월 {week}주차 시사상식.pdf'    

convert(input_file, output_file)

print(f'{month}월 {week}주차 상식 파일 생성 완료')


100%|██████████| 1/1 [00:04<00:00,  4.33s/it]

6월 1주차 상식 파일 생성 완료
